In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/estc_processed.csv')

/home/konstaku/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (67) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
columns = np.array([
 'control_number', 'system_control_number', 'author_name', 'author_birth', 'author_death', 'title', 'publication_year_from',
 'publication_year_till',
 'volnumber',
 'volcount',
 'parts',
 'publisher',
 'publication_place',
 'author_pseudonyme',
 'author',
 'publication_year',
 'publication_decade',
 'id',
])

df2 = df[columns]
df2 = df2.set_index('system_control_number')

df_canon = pd.read_csv('../data/estc_metadata_work_id_and_canon_status.csv')

df_canon = df_canon[df_canon['is_canon'] == True]
df_canon = df_canon.set_index('system_control_number')
df_canon = df_canon.join(df2)

In [4]:
# Create texts
df_final = df_canon[['id', 'title']]
df_final.to_csv('../data/texts.csv')
text_ids = set(list(df_canon['id']))

In [5]:
# Create actors
df_actors = pd.read_csv('../data/unified_actors.csv')
df_actor_details = pd.read_csv('../data/unified_actorlinks_enriched.csv')
df_actor_details = df_actor_details.set_index('curives')

actor_ids = set(df_actors['actor_id'])
df_actors = df_actors[['actor_id', 'name_unified']]
names_by_id = df_actor_details[['actor_id', 'actor_name_primary']].set_index('actor_id').to_dict()['actor_name_primary']

def get_name(val):
    return names_by_id.get(val, 'N/A')

df_actors['name'] = df_actors['actor_id'].apply(get_name)
df_actors = df_actors.set_index('actor_id')[['name']]
df_actors.to_csv('../data/actors.csv')

/home/konstaku/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (2,3,7,8,9,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df_canon_join = df_canon[['id']].join(df_actor_details)[['actor_id', 'id', 'actor_role_author', 'actor_role_printer', 'actor_role_publisher', 'actor_role_bookseller', 'actor_role_corporate_author', 'actor_role_geographic_record', 'actor_role_unknown', 'actor_role_translator', 'actor_role_editor', 'actor_role_engraver', 'actor_role_other', 'pubyear', 'publoc']]

In [7]:
entity_connections = []
processed_ids = set([])
years, places = set([]), set([])
year_connections = []
place_connections = []

roles = ['actor_role_author', 'actor_role_printer', 'actor_role_publisher', 'actor_role_bookseller', 'actor_role_corporate_author', 'actor_role_geographic_record', 'actor_role_unknown', 'actor_role_translator', 'actor_role_editor', 'actor_role_engraver', 'actor_role_other'] 
connections = ['author', 'printer', 'publisher', 'bookseller', 'corporate author', 'geographic record', 'unknown' , 'translator', 'editor', 'engraver', 'other']
stats = ['pubyear', 'publoc']

def process_connections(row):
    for role, connection in zip(roles, connections):
        if row[role]:
            entity_connections.append({'text': row['id'], 'entity': row['actor_id'], 'type': connection})
        if row['id'] not in processed_ids:
            processed_ids.add(row['id'])
            if row.get('publoc'):
                places.add(row['publoc'])
                place_connections.append({'text': row['id'], 'place': row['publoc']})
            if row.get('pubyear'):
                try:
                    years.add(row['pubyear'])
                    year_connections.append({'text': row['id'], 'year': int(row['pubyear'])})    
                except:
                    pass
            
for i, (_, row) in enumerate(df_canon_join.iterrows()):
    
    if i % 10000 == 5000:
        print(f'Progress at {i} / {len(df_canon_join)}')
    process_connections(row)

Progress at 5000 / 112065
Progress at 15000 / 112065
Progress at 25000 / 112065
Progress at 35000 / 112065
Progress at 45000 / 112065
Progress at 55000 / 112065
Progress at 65000 / 112065
Progress at 75000 / 112065
Progress at 85000 / 112065
Progress at 95000 / 112065
Progress at 105000 / 112065


In [8]:
year_df = pd.DataFrame.from_dict(year_connections).set_index('text')
place_df = pd.DataFrame.from_dict(place_connections).set_index('text')
entity_df = pd.DataFrame.from_dict(entity_connections).set_index('text')

year_df.to_csv('../data/year_connections.csv')
place_df.to_csv('../data/place_connections.csv')
entity_df.to_csv('../data/entity_connections.csv')

In [17]:
text_pairs = pd.read_csv('../data/idpairs.csv')
pair_by_fulltext_id = text_pairs.set_index('fulltext_id').to_dict()['curives_id']
fulltext_ids = (set([str(x) for x in text_pairs['fulltext_id']]))

In [ ]:
import json
import time
import os

json_files = [c for c in os.listdir('../data/clusters') if not c.endswith('.gz') and 'ipynb_checkpoints' not in c]

# np.random.shuffle(json_files)
def create_connections():
    start = time.time()
    clusters = {}
    for i, json_path in enumerate(json_files):
        with open('../data/clusters/' + json_path) as f:
            data = json.load(f)
            for cluster, reuses in data.items():
                reuse_arr = []
                for entry in reuses.get('hits', []):
                    id_ref = entry['doc_id'].split('/')[-1].split('.')[0]
                    if id_ref in fulltext_ids:
                        try:
                            curive = pair_by_fulltext_id[int(id_ref)]
                            reuse_arr.append({'text_id': id_ref, 'reuse_id': entry['id'], 'curives_id': curive})
                        except:
                            pass

                if len(reuse_arr) > 1:
                    clusters[cluster] = reuse_arr

        if i % 1000 == 999:
            print(f'Time for last 1000 cluster files: {time.time() - start}' f' Progress: {i + 1} / {len(json_files)}')
            start = time.time()
    return clusters
clusters = create_connections()

Time for last 1000 cluster files: 62.10176467895508 Progress: 1000 / 28319
Time for last 1000 cluster files: 66.8294587135315 Progress: 2000 / 28319
Time for last 1000 cluster files: 68.3068585395813 Progress: 3000 / 28319
Time for last 1000 cluster files: 67.07979726791382 Progress: 4000 / 28319
Time for last 1000 cluster files: 68.19957613945007 Progress: 5000 / 28319
Time for last 1000 cluster files: 67.92010760307312 Progress: 6000 / 28319
Time for last 1000 cluster files: 66.44391870498657 Progress: 7000 / 28319
Time for last 1000 cluster files: 66.11498737335205 Progress: 8000 / 28319
Time for last 1000 cluster files: 64.97461605072021 Progress: 9000 / 28319
Time for last 1000 cluster files: 63.28676247596741 Progress: 10000 / 28319


In [57]:
import time

year_by_id = year_df.to_dict()['year']

reuse_connections = {}

start = time.time()
def create_connections(cluster, cid):
    if len(cluster) == 1:
        return
    
    cluster = list(filter(lambda c: year_by_id.get(c['curives_id'], -11) == year_by_id.get(c['curives_id'], -22), cluster))
    uniques_texts = set([c['curives_id'] for c in cluster])
    cs = sorted([{'text_id': t_id, 'year': int(year_by_id[t_id])} for t_id in uniques_texts], key=lambda x: -x['year'])
    
    for i in range(len(cs) - 1):
        for j in range(i + 1, len(cs)):
            start = cs[i]
            end = cs[j]
            start_id = start['text_id']
            end_id = end['text_id']
            reuse_connections[(start_id, end_id)] = reuse_connections.get((start_id, end_id), 0) + 1
            if start['year'] == end['year']:
                reuse_connections[(end_id, start_id)] = reuse_connections.get((end_id, start_id), 0) + 1

for i, (k, v) in enumerate(clusters.items()):
    create_connections(v, k)
    if i % 1_000_000 == 500_000:
        print(f'Time for last 1_000_000 cluster files: {time.time() - start}' f' Progress: {i + 1} / {len(clusters)}')
        start = time.time()
    

Time for last 1_000_000 cluster files: 15.4519784450531 Progress: 500001 / 10479850
Time for last 1_000_000 cluster files: 33.522534132003784 Progress: 1500001 / 10479850
Time for last 1_000_000 cluster files: 37.305570125579834 Progress: 2500001 / 10479850
Time for last 1_000_000 cluster files: 32.919373512268066 Progress: 3500001 / 10479850
Time for last 1_000_000 cluster files: 34.43808937072754 Progress: 4500001 / 10479850
Time for last 1_000_000 cluster files: 33.277485609054565 Progress: 5500001 / 10479850
Time for last 1_000_000 cluster files: 22.94879722595215 Progress: 6500001 / 10479850
Time for last 1_000_000 cluster files: 21.82454514503479 Progress: 7500001 / 10479850
Time for last 1_000_000 cluster files: 28.396111965179443 Progress: 8500001 / 10479850
Time for last 1_000_000 cluster files: 38.824570655822754 Progress: 9500001 / 10479850


In [59]:
data_arr = [{'source': p[0], 'reuser': p[1], 'count': num} for p, num in reuse_connections.items()]

conn_df = pd.DataFrame(data_arr)

NameError: name 'reuse_connections' is not defined

In [76]:
conn_df.to_csv('../data/cluster_connections_reduced.csv')

In [41]:
authors = entity_df[entity_df['type'] == 'author'].join(df_actors, on='entity')

In [58]:
joined_df = df_final.set_index('id').join(year_df).join(place_df).join(authors)[['title', 'year', 'place', 'name']]

joined_df.to_csv('../data/text_with_details.csv', sep='|')

In [54]:
set(joined_df['year'])

{nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 1473.0,
 1474.0,
 1476.0,
 1477.0,
 1479.0,
 1481.0,
 1482.0,
 1483.0,
 1484.0,
 1485.0,
 1488.0,
 1490.0,
 1492.0,
 1494.0,
 1495.0,
 1496.0,
 1497.0,
 1498.0,
 1499.0,
 1500.0,
 1501.0,
 1502.0,
 1503.0,
 1504.0,
 1505.0,
 1506.0,
 1507.0,
 1508.0,
 1509.0,
 1510.0,
 1511.0,
 1512.0,
 1513.0,
 1514.0,
 1515.0,
 1516.0,
 1517.0,
 1518.0,
 1519.0,
 1520.0,
 1521.0,
 1522.0,
 1523.0,
 1524.0,
 1525.0,
 1526.0,
 1527.0,
 1528.0,
 1529.0,
 1530.0,
 1531.0,
 1532.0,
 1533.0,
 1534.0,
 1535.0,
 1536.0,
 1537.0,
 

In [ ]:
year_df = pd.DataFrame.from_dict(year_connections).set_index('text')
place_df = pd.DataFrame.from_dict(place_connections).set_index('text')
entity_df = pd.DataFrame.from_dict(entity_connections).set_index('text')
